In [26]:
from openai import OpenAI
import json
from lightrag.utils import setup_env

client = OpenAI()

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    print(response)
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
print(run_conversation())

ChatCompletion(id='chatcmpl-9ejxuyW8plusyfYrsycw5tB8E41yo', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_O9SuzUHvcbI2pc5ExAiOxV97', function=Function(arguments='{"location": "San Francisco, CA"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_MMp8a5gYKLULm4jdJYRO6VVP', function=Function(arguments='{"location": "Tokyo, Japan"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_tjMDtDziuS3DA2ZTaV5y9Vic', function=Function(arguments='{"location": "Paris, France"}', name='get_current_weather'), type='function')]))], created=1719496838, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_ce0793330f', usage=CompletionUsage(completion_tokens=68, prompt_tokens=85, total_tokens=153))
ChatCompletion(id='chatcmpl-9ejxwzAPWC8H

In [27]:
from dataclasses import dataclass
from typing import Any, Dict, List, Tuple
import numpy as np
import time
import asyncio

def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    time.sleep(1)
    return a * b

def add(a: int, b: int) -> int:
    """Add two numbers."""
    time.sleep(1)
    return a + b

async def divide(a: float, b: float) -> float:
    """Divide two numbers."""
    await asyncio.sleep(1)
    return float(a) / b


async def search(query: str) -> List[str]:
    """Search for query and return a list of results."""
    await asyncio.sleep(1)
    return ["result1" + query, "result2" + query]


def numpy_sum(arr: np.ndarray) -> float:
    """Sum the elements of an array."""
    return np.sum(arr)

x = 2
@dataclass
class Point:
    x: int
    y: int

def add_points(p1: Point, p2: Point) -> Point:
    return Point(p1.x + p2.x, p1.y + p2.y)

all_functions = [multiply, add, divide, search, numpy_sum, add_points]

all_functions_dict = {f.__name__: f for f in all_functions}

In [29]:
# describing the functions

from lightrag.core.func_tool import FunctionTool


tools = [
    FunctionTool(fn=fn) for fn in [multiply, add, divide, search, numpy_sum, add_points]
]
for tool in tools:
    print(tool)

FunctionTool(fn: <function multiply at 0x14d9d3f60>, async: False, definition: FunctionDefinition(func_name='multiply', func_desc='multiply(a: int, b: int) -> int\nMultiply two numbers.', func_parameters={'type': 'object', 'properties': {'a': {'type': 'int'}, 'b': {'type': 'int'}}, 'required': ['a', 'b']}))
FunctionTool(fn: <function add at 0x14d9e4040>, async: False, definition: FunctionDefinition(func_name='add', func_desc='add(a: int, b: int) -> int\nAdd two numbers.', func_parameters={'type': 'object', 'properties': {'a': {'type': 'int'}, 'b': {'type': 'int'}}, 'required': ['a', 'b']}))
FunctionTool(fn: <function divide at 0x14d9e40e0>, async: True, definition: FunctionDefinition(func_name='divide', func_desc='divide(a: float, b: float) -> float\nDivide two numbers.', func_parameters={'type': 'object', 'properties': {'a': {'type': 'float'}, 'b': {'type': 'float'}}, 'required': ['a', 'b']}))
FunctionTool(fn: <function search at 0x14d9e4180>, async: True, definition: FunctionDefiniti

In [32]:
# tool definition for get_current_weather

ft = FunctionTool(fn=get_current_weather)
ft.definition.to_dict()

{'func_name': 'get_current_weather',
 'func_desc': "get_current_weather(location, unit='fahrenheit')\nGet the current weather in a given location",
 'func_parameters': {'type': 'object',
  'properties': {'location': {'type': 'Any'},
   'unit': {'type': 'Any', 'default': 'fahrenheit'}},
  'required': ['location']}}

In [2]:
from lightrag.core.tool_helper import FunctionTool

add_tool = FunctionTool(fn=add)
print(add_tool)
async_search_tool = FunctionTool(fn=search)
print(async_search_tool)

add_points_tool = FunctionTool(fn=add_points)
print(add_points_tool)

numpy_sum_tool = FunctionTool(fn=numpy_sum)
print(numpy_sum_tool)

FunctionTool(fn: <function add at 0x1108705e0>, async: False, definition: FunctionDefinition(func_name='add', func_desc='Add two numbers.', func_parameters={'type': 'object', 'properties': {'a': {'type': 'int'}, 'b': {'type': 'int'}}, 'required': ['a', 'b']}))
FunctionTool(fn: <function search at 0x11081fec0>, async: True, definition: FunctionDefinition(func_name='search', func_desc='Search for query and return a list of results.', func_parameters={'type': 'object', 'properties': {'query': {'type': 'str'}}, 'required': ['query']}))
FunctionTool(fn: <function add_points at 0x11190e5c0>, async: False, definition: FunctionDefinition(func_name='add_points', func_desc='None', func_parameters={'type': 'object', 'properties': {'p1': {'type': 'Point', 'properties': {'x': {'type': 'int'}, 'y': {'type': 'int'}}, 'required': ['x', 'y']}, 'p2': {'type': 'Point', 'properties': {'x': {'type': 'int'}, 'y': {'type': 'int'}}, 'required': ['x', 'y']}}, 'required': ['p1', 'p2']}))
FunctionTool(fn: <funct

In [3]:
# prepare a template for generator
template = r"""<SYS>You have these tools available:
{% if tools %}
<TOOLS>
{% for tool in tools %}
{{ loop.index }}.
{{tool}}
------------------------
{% endfor %}
</TOOLS>
{% endif %}
<OUTPUT_FORMAT>
{{output_format_str}}
</OUTPUT_FORMAT>
</SYS>
User: {{input_str}}
You:
"""

In [4]:
# let's see how the template can be rendered with tools
from lightrag.core.prompt_builder import Prompt

prompt = Prompt(template=template)
tools = [FunctionTool(fn=f) for f in all_functions]

# pass tools's definition to the prompt

renered_prompt = prompt(tools=[tool.definition.to_yaml() for tool in tools])
print(renered_prompt)


<SYS>You have these tools available:
<TOOLS>
1.
func_name: multiply
func_desc: Multiply two numbers.
func_parameters:
  type: object
  properties:
    a:
      type: int
    b:
      type: int
  required:
  - a
  - b

------------------------
2.
func_name: add
func_desc: Add two numbers.
func_parameters:
  type: object
  properties:
    a:
      type: int
    b:
      type: int
  required:
  - a
  - b

------------------------
3.
func_name: divide
func_desc: Divide two numbers.
func_parameters:
  type: object
  properties:
    a:
      type: float
    b:
      type: float
  required:
  - a
  - b

------------------------
4.
func_name: search
func_desc: Search for query and return a list of results.
func_parameters:
  type: object
  properties:
    query:
      type: str
  required:
  - query

------------------------
5.
func_name: numpy_sum
func_desc: Sum the elements of an array.
func_parameters:
  type: object
  properties:
    arr:
      type: ndarray
  required:
  - arr

----------

In [5]:
# let's render the output format using Function class 

from lightrag.core.types import Function

prompt = Prompt(template=template)

output_data_class = Function 
output_format_str = output_data_class.to_yaml_signature(exclude=["thought"])

renered_prompt= prompt(output_format_str=output_format_str)
print(renered_prompt)


<SYS>You have these tools available:
<OUTPUT_FORMAT>
name: The name of the function (str) (optional)
args: The positional arguments of the function (Optional) (optional)
kwargs: The keyword arguments of the function (Optional) (optional)
</OUTPUT_FORMAT>
</SYS>
User: None
You:



In [6]:
# let's adds more instruction and this time, we will use JsonOutputParser

from lightrag.components.output_parsers import JsonOutputParser

func_parser = JsonOutputParser(data_class=Function)
instructions = func_parser.format_instructions(exclude=["thought"])
print(instructions)


Your output should be formatted as a standard JSON instance with the following schema:
```
{
    "name": "The name of the function (str) (optional)",
    "args": "The positional arguments of the function (Optional) (optional)",
    "kwargs": "The keyword arguments of the function (Optional) (optional)"
}
```
-Make sure to always enclose the JSON output in triple backticks (```). Please do not add anything other than valid JSON output!
-Use double quotes for the keys and string values.
-Follow the JSON formatting conventions.


In [7]:
# create the generator
from lightrag.core.generator import Generator
from lightrag.core.types import ModelClientType

model_kwargs = {"model": "gpt-3.5-turbo"}
prompt_kwargs = {
        "tools": [tool.definition.to_yaml() for tool in tools],
        "output_format_str": func_parser.format_instructions(exclude=["thought", "args"])
    }
# only use name and kwargs for the output data class
generator = Generator(
    model_client=ModelClientType.OPENAI(),
    model_kwargs=model_kwargs,
    template=template,
    prompt_kwargs=prompt_kwargs,
    output_processors=func_parser
)
generator

Generator(
  model_kwargs={'model': 'gpt-3.5-turbo'}, 
  (prompt): Prompt(
    template: <SYS>You have these tools available:
    {% if tools %}
    <TOOLS>
    {% for tool in tools %}
    {{ loop.index }}.
    {{tool}}
    ------------------------
    {% endfor %}
    </TOOLS>
    {% endif %}
    <OUTPUT_FORMAT>
    {{output_format_str}}
    </OUTPUT_FORMAT>
    </SYS>
    User: {{input_str}}
    You:
    , prompt_kwargs: {'tools': ['func_name: multiply\nfunc_desc: Multiply two numbers.\nfunc_parameters:\n  type: object\n  properties:\n    a:\n      type: int\n    b:\n      type: int\n  required:\n  - a\n  - b\n', 'func_name: add\nfunc_desc: Add two numbers.\nfunc_parameters:\n  type: object\n  properties:\n    a:\n      type: int\n    b:\n      type: int\n  required:\n  - a\n  - b\n', 'func_name: divide\nfunc_desc: Divide two numbers.\nfunc_parameters:\n  type: object\n  properties:\n    a:\n      type: float\n    b:\n      type: float\n  required:\n  - a\n  - b\n', 'func_name: searc

In [8]:
arr = np.array([[1, 2], [3, 4]])
numpy_sum(arr)

10

In [9]:
queries = ["add 2 and 3", "search for something", "add points (1, 2) and (3, 4)", "sum numpy array with arr = np.array([[1, 2], [3, 4]])", "multiply 2 with local variable x", "divide 2 by 3"]

for query in queries:
    prompt_kwargs = {"input_str": query}
    print(f"Query: {query}")
    try:
        result = generator(prompt_kwargs=prompt_kwargs)
        print(result)

        func = Function.from_dict(result.data)

        print(func)
        assert isinstance(func, Function), f"Expected Function, got {type(result.data)}"
        fun_output = all_functions_dict[func.name](*func.args, **func.kwargs) 
        print(f"Function output: {fun_output}")
    except Exception as e:
        print(e)
        print(f"Failed to execute the function for query: {query}, func: {result.data}, error: {e}")

Query: add 2 and 3
GeneratorOutput(data={'name': 'add', 'kwargs': {'a': 2, 'b': 3}}, error=None, usage=None, raw_response='{\n    "name": "add",\n    "kwargs": {\n        "a": 2,\n        "b": 3\n    }\n}')
Function(thought=None, name='add', args=[], kwargs={'a': 2, 'b': 3})
Function output: 5
Query: search for something
GeneratorOutput(data={'name': 'search', 'kwargs': {'query': 'something'}}, error=None, usage=None, raw_response='```\n{\n    "name": "search",\n    "kwargs": {\n        "query": "something"\n    }\n}\n```')
Function(thought=None, name='search', args=[], kwargs={'query': 'something'})
Function output: <coroutine object search at 0x122725d80>
Query: add points (1, 2) and (3, 4)
GeneratorOutput(data={'name': 'add_points', 'kwargs': {'p1': {'x': 1, 'y': 2}, 'p2': {'x': 3, 'y': 4}}}, error=None, usage=None, raw_response='```json\n{\n    "name": "add_points",\n    "kwargs": {\n        "p1": {\n            "x": 1,\n            "y": 2\n        },\n        "p2": {\n            

Error processing the output: Error: Got invalid JSON object. Error: while constructing a mapping
  in "<unicode string>", line 3, column 15:
        "kwargs": {
                  ^
found unhashable key
  in "<unicode string>", line 4, column 24:
            "arr": [1, 2], [3, 4]
                           ^. Got JSON string: {
    "name": "numpy_sum",
    "kwargs": {
        "arr": [1, 2], [3, 4]
    }
}


Trying to fix potential missing commas...
Trying to fix scaped single quotes...
Fixed JSON string: {
    "name": "numpy_sum",
    "kwargs": {
        "arr": [1, 2], [3, 4]
    }
}
Parsing JSON string with PyYAML...
GeneratorOutput(data=None, error='Error: Got invalid JSON object. Error: while constructing a mapping\n  in "<unicode string>", line 3, column 15:\n        "kwargs": {\n                  ^\nfound unhashable key\n  in "<unicode string>", line 4, column 24:\n            "arr": [1, 2], [3, 4]\n                           ^. Got JSON string: {\n    "name": "numpy_sum",\n    "kwargs": {\n        "arr": [1, 2], [3, 4]\n    }\n}', usage=None, raw_response='```\n{\n    "name": "numpy_sum",\n    "kwargs": {\n        "arr": [[1, 2], [3, 4]]\n    }\n}\n```')
'NoneType' object has no attribute 'items'
Failed to execute the function for query: sum numpy array with arr = np.array([[1, 2], [3, 4]]), func: None, error: 'NoneType' object has no attribute 'items'
Query: multiply 2 with local v

/var/folders/h8/nhgbdr4d18x2r49j4pk5z6gw0000gn/T/ipykernel_30203/3669837920.py:14: RuntimeWarning: coroutine 'search' was never awaited
  fun_output = all_functions_dict[func.name](*func.args, **func.kwargs)


GeneratorOutput(data={'name': 'divide', 'kwargs': {'a': 2.0, 'b': 3.0}}, error=None, usage=None, raw_response='```json\n{\n    "name": "divide",\n    "kwargs": {\n        "a": 2.0,\n        "b": 3.0\n    }\n}\n```')
Function(thought=None, name='divide', args=[], kwargs={'a': 2.0, 'b': 3.0})
Function output: 0.6666666666666666


Problems with Function directly:
1. difficult to support data types. Unless to update the function to use dict version of the data types to do it.

```python
def add_points(p1: dict, p2: dict) -> dict:
    p1 = Point(**p1)
    p2 = Point(**p2)
    return add_points_tool.fn(p1, p2).__dict__
```
2. difficult to use variable as arguments. [TODO: find a proper way to demonstrate it]

In [10]:
# let's use FunctionExpression to call the function instead 

from lightrag.core.types import FunctionExpression

output_data_class = FunctionExpression
output_format_str = output_data_class.to_yaml_signature(exclude=["thought"])
print(output_format_str)

# lets' add one example to be more robust that they should call it with function call expression
example = FunctionExpression.from_function(thought=None, func=add_points, **{"p1": Point(1, 2), "p2": Point(3, 4)})
print(example)

action: Formatted as FuncName(<args>, <kwargs>), where FuncName is the function name, <args> are positional arguments, and <kwargs> are keyword arguments in key=value form. Example: 'FuncName(1, b=2)' calls 'FuncName' with positional argument 1 and keyword argument b=2. (str) (required)
FunctionExpression(thought=None, action='add_points(p1=Point(x=1, y=2), p2=Point(x=3, y=4))')


In [11]:
# also use json output parser and create a new generator

parser = JsonOutputParser(data_class=FunctionExpression, example=example)
instructions = parser.format_instructions(exclude=["thought"])

prompt_kwargs = {
        "tools": [tool.definition.to_yaml() for tool in tools],
        "output_format_str": parser.format_instructions(exclude=["thought"]),
    }
generator = Generator(
    model_client=ModelClientType.OPENAI(),
    model_kwargs=model_kwargs,
    template=template,
    prompt_kwargs=prompt_kwargs,
    output_processors=parser
)

generator.print_prompt(**prompt_kwargs)

Prompt:

<SYS>You have these tools available:
<TOOLS>
1.
func_name: multiply
func_desc: Multiply two numbers.
func_parameters:
  type: object
  properties:
    a:
      type: int
    b:
      type: int
  required:
  - a
  - b

------------------------
2.
func_name: add
func_desc: Add two numbers.
func_parameters:
  type: object
  properties:
    a:
      type: int
    b:
      type: int
  required:
  - a
  - b

------------------------
3.
func_name: divide
func_desc: Divide two numbers.
func_parameters:
  type: object
  properties:
    a:
      type: float
    b:
      type: float
  required:
  - a
  - b

------------------------
4.
func_name: search
func_desc: Search for query and return a list of results.
func_parameters:
  type: object
  properties:
    query:
      type: str
  required:
  - query

------------------------
5.
func_name: numpy_sum
func_desc: Sum the elements of an array.
func_parameters:
  type: object
  properties:
    arr:
      type: ndarray
  required:
  - arr

-

In [12]:
import ast
import builtins
import contextlib
import ctypes
import sys
import threading
import time

# Define a list of safe built-ins
SAFE_BUILTINS = {
    'abs': abs,
    'all': all,
    'any': any,
    'bin': bin,
    'bool': bool,
    'bytearray': bytearray,
    'bytes': bytes,
    'callable': callable,
    'chr': chr,
    'complex': complex,
    'dict': dict,
    'divmod': divmod,
    'enumerate': enumerate,
    'filter': filter,
    'float': float,
    'format': format,
    'frozenset': frozenset,
    'getattr': getattr,
    'hasattr': hasattr,
    'hash': hash,
    'hex': hex,
    'int': int,
    'isinstance': isinstance,
    'issubclass': issubclass,
    'iter': iter,
    'len': len,
    'list': list,
    'map': map,
    'max': max,
    'min': min,
    'next': next,
    'object': object,
    'oct': oct,
    'ord': ord,
    'pow': pow,
    'range': range,
    'repr': repr,
    'reversed': reversed,
    'round': round,
    'set': set,
    'slice': slice,
    'sorted': sorted,
    'str': str,
    'sum': sum,
    'tuple': tuple,
    'type': type,
    'zip': zip,
}

# Define a context manager to limit execution time
# Create a sandbox execution function
def sandbox_exec(code, context=SAFE_BUILTINS, timeout=5):

    try:
        compiled_code = compile(code, '<string>', 'exec')

        # Result dictionary to store execution results
        result = {
            "output" : None,
            "error" : None
        }

        # Define a target function for the thread
        def target():
            try:
                # Execute the code
                exec(compiled_code, context, result)
            except Exception as e:
                result["error"] = e
            

        # Create a thread to execute the code
        thread = threading.Thread(target=target)
        thread.start()
        thread.join(timeout)

        # Check if the thread is still alive (timed out)
        if thread.is_alive():
            result["error"] = TimeoutError("Execution timed out")
            raise TimeoutError("Execution timed out")
    except Exception as e:
        print(f"Errpr at sandbox_exec: {e}")
        raise e

    return result

# Example usage
code = """
def add(a, b+5):
    return a + b

output = add(1, 2+y)
"""

try:
    result = sandbox_exec(code)
    print("Sandbox output:", result)
except TimeoutError as e:
    print(e)
except Exception as e:
    print("Sandbox error:", e)


Errpr at sandbox_exec: invalid syntax (<string>, line 2)
Sandbox error: invalid syntax (<string>, line 2)


In [17]:
# run the generator but we will use FunctionTool.parse_function_call_expr and have a context map 

all_functions_dict.update(
    {
    "Point": Point,
    # support numpy
    "np": np,
    "np.ndarray": np.ndarray,
    "array": np.array,
    "arr": arr,
    "np.array": np.array,
    "x": x
    }
)
y=4
print(all_functions_dict)
for query in queries+["Add 5 to variable y"]:

    try:
        print(f"Query: {query}")
        prompt_kwargs = {"input_str": query}
        result = generator(prompt_kwargs=prompt_kwargs)
        print(result)

        func_expr = FunctionExpression.from_dict(result.data)

        print(func_expr)
        assert isinstance(func_expr, FunctionExpression), f"Expected FunctionExpression, got {type(result.data)}"

        # more secure way to handle function call
        func: Function = FunctionTool.parse_function_call_expr(expr=func_expr.action, context_map=all_functions_dict)
        print(func)
        fun_output = all_functions_dict[func.name](*func.args, **func.kwargs)
        print("func output:", fun_output)

        print(f"func expr: {func_expr.action}")

        # eval without security check by using eval directly
        # less secure but even more powerful and flexible
        fun_output = eval(func_expr.action)
        print("func output:", fun_output)

        # sandbox_exec
        action = "output=" + func_expr.action
        result = sandbox_exec(action, context={**SAFE_BUILTINS, **all_functions_dict})
        print("sandbox output:", result)
    except Exception as e:
        print(e)
        print(f"Failed to execute the function for query: {query}, func: {result.data}, error: {e}")
        try:
            fun_output = eval(func_expr.action)
            print("func output:", fun_output)

            #sandbox_exec
            action = "output=" + func_expr.action
            result = sandbox_exec(action, context={**SAFE_BUILTINS, **all_functions_dict})
            print("sandbox output:", result)
        except Exception as e:
            print(e)
            print(f"Failed to execute the function for query: {query}, func: {result.data}, error: {e}")

{'multiply': <function multiply at 0x110990900>, 'add': <function add at 0x1108705e0>, 'divide': <function divide at 0x11081ff60>, 'search': <function search at 0x11081fec0>, 'numpy_sum': <function numpy_sum at 0x11190c540>, 'add_points': <function add_points at 0x11190e5c0>, 'Point': <class '__main__.Point'>, 'np': <module 'numpy' from '/Users/liyin/Documents/test/LightRAG/.venv/lib/python3.11/site-packages/numpy/__init__.py'>, 'np.ndarray': <class 'numpy.ndarray'>, 'array': <built-in function array>, 'arr': array([[1, 2],
       [3, 4]]), 'np.array': <built-in function array>, 'x': 2}
Query: add 2 and 3
GeneratorOutput(data={'action': 'add(2, b=3)'}, error=None, usage=None, raw_response='{\n    "action": "add(2, b=3)"\n}')
FunctionExpression(thought=None, action='add(2, b=3)')
Function(thought=None, name='add', args=[2], kwargs={'b': 3})
func output: 5
func expr: add(2, b=3)
func output: 5
sandbox output: {'output': 5, 'error': None}
Query: search for something
GeneratorOutput(data={

/var/folders/h8/nhgbdr4d18x2r49j4pk5z6gw0000gn/T/ipykernel_30203/1940296491.py:40: RuntimeWarning: coroutine 'search' was never awaited
  fun_output = eval(func_expr.action)
/var/folders/h8/nhgbdr4d18x2r49j4pk5z6gw0000gn/T/ipykernel_30203/1940296491.py:22: RuntimeWarning: coroutine 'search' was never awaited
  result = generator(prompt_kwargs=prompt_kwargs)
/var/folders/h8/nhgbdr4d18x2r49j4pk5z6gw0000gn/T/ipykernel_30203/1940296491.py:33: RuntimeWarning: coroutine 'search' was never awaited
  fun_output = all_functions_dict[func.name](*func.args, **func.kwargs)


GeneratorOutput(data={'action': 'add_points(p1=Point(x=1, y=2), p2=Point(x=3, y=4))'}, error=None, usage=None, raw_response='```\n{\n    "action": "add_points(p1=Point(x=1, y=2), p2=Point(x=3, y=4))"\n}\n```')
FunctionExpression(thought=None, action='add_points(p1=Point(x=1, y=2), p2=Point(x=3, y=4))')
Function(thought=None, name='add_points', args=[], kwargs={'p1': Point(x=1, y=2), 'p2': Point(x=3, y=4)})
func output: Point(x=4, y=6)
func expr: add_points(p1=Point(x=1, y=2), p2=Point(x=3, y=4))
func output: Point(x=4, y=6)
sandbox output: {'output': Point(x=4, y=6), 'error': None}
Query: sum numpy array with arr = np.array([[1, 2], [3, 4]])


Error Field elements must be 2- or 3-tuples, got '3' parsing function call expression: numpy_sum(arr=np.array([1, 2], [3, 4]))


GeneratorOutput(data={'action': 'numpy_sum(arr=np.array([1, 2], [3, 4]))'}, error=None, usage=None, raw_response='{\n    "action": "numpy_sum(arr=np.array([[1, 2], [3, 4]]))"\n}')
FunctionExpression(thought=None, action='numpy_sum(arr=np.array([1, 2], [3, 4]))')
Error Field elements must be 2- or 3-tuples, got '3' parsing function call expression: numpy_sum(arr=np.array([1, 2], [3, 4]))
Failed to execute the function for query: sum numpy array with arr = np.array([[1, 2], [3, 4]]), func: {'action': 'numpy_sum(arr=np.array([1, 2], [3, 4]))'}, error: Error Field elements must be 2- or 3-tuples, got '3' parsing function call expression: numpy_sum(arr=np.array([1, 2], [3, 4]))
Field elements must be 2- or 3-tuples, got '3'
Failed to execute the function for query: sum numpy array with arr = np.array([[1, 2], [3, 4]]), func: {'action': 'numpy_sum(arr=np.array([1, 2], [3, 4]))'}, error: Field elements must be 2- or 3-tuples, got '3'
Query: multiply 2 with local variable x
GeneratorOutput(dat

Error Error: 'y', y does not exist in the context_map. parsing function call expression: add(a=5, b=y)


GeneratorOutput(data={'action': 'add(a=5, b=y)'}, error=None, usage=None, raw_response='{\n    "action": "add(a=5, b=y)"\n}')
FunctionExpression(thought=None, action='add(a=5, b=y)')
Error Error: 'y', y does not exist in the context_map. parsing function call expression: add(a=5, b=y)
Failed to execute the function for query: Add 5 to variable y, func: {'action': 'add(a=5, b=y)'}, error: Error Error: 'y', y does not exist in the context_map. parsing function call expression: add(a=5, b=y)
func output: 9
sandbox output: {'output': None, 'error': NameError("name 'y' is not defined")}


Multiple function calls

In [18]:
multple_function_call_template = r"""<SYS>You have these tools available:
{% if tools %}
<TOOLS>
{% for tool in tools %}
{{ loop.index }}.
{{tool}}
------------------------
{% endfor %}
</TOOLS>
{% endif %}
<OUTPUT_FORMAT>
Here is how you call one function.
{{output_format_str}}
Return a List using `[]` of the above JSON objects. You can have length of 1 or more.
Do not call multiple functions in one action field.
</OUTPUT_FORMAT>
<SYS>
{{input_str}}
You:
"""

In [19]:
queries = ["add 2 and 3", "search for something", "add points (1, 2) and (3, 4)", "sum numpy array with arr = np.array([[1, 2], [3, 4]])", "multiply 2 with local variable x", "divide 2 by 3"]

from lightrag.components.output_parsers import ListOutputParser
from lightrag.core.string_parser import JsonParser # improve a list of json

preset_prompt_kwargs = {
        "tools": [tool.definition.to_yaml() for tool in tools],
        "output_format_str": parser.format_instructions(exclude=["thought"])
    }
multi_call_gen = Generator(
    model_client=ModelClientType.OPENAI(),
    model_kwargs=model_kwargs,
    template=multple_function_call_template,
    prompt_kwargs=preset_prompt_kwargs,
    output_processors=JsonParser()
)
print(multi_call_gen)
multi_call_gen.print_prompt()

Generator(
  model_kwargs={'model': 'gpt-3.5-turbo'}, 
  (prompt): Prompt(
    template: <SYS>You have these tools available:
    {% if tools %}
    <TOOLS>
    {% for tool in tools %}
    {{ loop.index }}.
    {{tool}}
    ------------------------
    {% endfor %}
    </TOOLS>
    {% endif %}
    <OUTPUT_FORMAT>
    Here is how you call one function.
    {{output_format_str}}
    Return a List using `[]` of the above JSON objects. You can have length of 1 or more.
    Do not call multiple functions in one action field.
    </OUTPUT_FORMAT>
    <SYS>
    {{input_str}}
    You:
    , prompt_kwargs: {'tools': ['func_name: multiply\nfunc_desc: Multiply two numbers.\nfunc_parameters:\n  type: object\n  properties:\n    a:\n      type: int\n    b:\n      type: int\n  required:\n  - a\n  - b\n', 'func_name: add\nfunc_desc: Add two numbers.\nfunc_parameters:\n  type: object\n  properties:\n    a:\n      type: int\n    b:\n      type: int\n  required:\n  - a\n  - b\n', 'func_name: divide\nfunc

In [24]:
def execute_function_by_parsing(func_expr: FunctionExpression, all_functions_dict: Dict[str, Any]) -> Any:
    func: Function = FunctionTool.parse_function_call_expr(expr=func_expr.action, context_map=all_functions_dict)
    print(func)
    fun_output = all_functions_dict[func.name](*func.args, **func.kwargs)
    print("func output:", fun_output)
    return fun_output


def execute_function_by_eval(func_expr: FunctionExpression) -> Any:

    print(f"func expr: {func_expr.action}")

    # eval without security check by using eval directly
    # less secure but even more powerful and flexible
    fun_output = eval(func_expr.action)
    print("func output:", fun_output)
    return fun_output

def execute_function_by_sandbox(func_expr: FunctionExpression, all_functions_dict: Dict[str, Any]) -> Any:
    # sandbox_exec
    action = "output=" + func_expr.action
    result = sandbox_exec(action, context={**SAFE_BUILTINS, **all_functions_dict})
    print("sandbox output:", result)

    return result




for i in range(0, len(queries), 2):
    query = " and ".join(queries[i:i+2])
    print(f"Query: {query}\n_________________________\n")
    prompt_kwargs = {"input_str": query}
    result = multi_call_gen(prompt_kwargs=prompt_kwargs)
    print(result)

    try:

        func_exprs = [FunctionExpression.from_dict(item) for item in result.data]

        print(func_exprs)
    except Exception as e:
        print(e)
        print(f"Failed to parse the function for query: {query}, func: {result.data}, error: {e}")
        continue
    try:
        func_outputs_1 = [execute_function_by_parsing(func_expr, all_functions_dict) for func_expr in func_exprs]
        print(f"fun_output by parsing: {func_outputs_1}\n_________________________\n")
    except Exception as e:
        print(e)
        print(f"Failed to execute the function for query: {query}, func: {result.data}, error: {e}")

    try:

        func_outputs_2 = [execute_function_by_eval(func_expr) for func_expr in func_exprs]
        print(f"fun_output by eval: {func_outputs_2}\n_________________________\n")
    except Exception as e:
        print(e)
        print(f"Failed to execute the function for query: {query}, func: {result.data}, error: {e}")

    try:

        func_outputs_3 = [execute_function_by_sandbox(func_expr, all_functions_dict) for func_expr in func_exprs]
        print(f"fun_output by sandbox: {func_outputs_3}\n_________________________\n")
    except Exception as e:
        print(e)
        print(f"Failed to execute the function for query: {query}, func: {result.data}, error: {e}")

        


Query: add 2 and 3 and search for something
_________________________

GeneratorOutput(data={'action': 'add(2, b=3)'}, error=None, usage=None, raw_response='```  \n{\n    "action": "add(2, b=3)"\n}\n{\n    "action": "search(query=\'something\')"\n}\n```')
'str' object has no attribute 'items'
Failed to parse the function for query: add 2 and 3 and search for something, func: {'action': 'add(2, b=3)'}, error: 'str' object has no attribute 'items'
Query: add points (1, 2) and (3, 4) and sum numpy array with arr = np.array([[1, 2], [3, 4]])
_________________________



Error Field elements must be 2- or 3-tuples, got '3' parsing function call expression: numpy_sum(arr=np.array([1, 2], [3, 4]))


GeneratorOutput(data=[{'action': 'add_points(p1=Point(x=1, y=2), p2=Point(x=3, y=4))'}, {'action': 'numpy_sum(arr=np.array([1, 2], [3, 4]))'}], error=None, usage=None, raw_response='```json\n[\n    {\n        "action": "add_points(p1=Point(x=1, y=2), p2=Point(x=3, y=4))"\n    },\n    {\n        "action": "numpy_sum(arr=np.array([[1, 2], [3, 4]]))"\n    }\n]\n```')
[FunctionExpression(thought=None, action='add_points(p1=Point(x=1, y=2), p2=Point(x=3, y=4))'), FunctionExpression(thought=None, action='numpy_sum(arr=np.array([1, 2], [3, 4]))')]
Function(thought=None, name='add_points', args=[], kwargs={'p1': Point(x=1, y=2), 'p2': Point(x=3, y=4)})
func output: Point(x=4, y=6)
Error Field elements must be 2- or 3-tuples, got '3' parsing function call expression: numpy_sum(arr=np.array([1, 2], [3, 4]))
Failed to execute the function for query: add points (1, 2) and (3, 4) and sum numpy array with arr = np.array([[1, 2], [3, 4]]), func: [{'action': 'add_points(p1=Point(x=1, y=2), p2=Point(x=

/var/folders/h8/nhgbdr4d18x2r49j4pk5z6gw0000gn/T/ipykernel_30203/2050537243.py:47: RuntimeWarning: coroutine 'search' was never awaited
  func_outputs_1 = [execute_function_by_parsing(func_expr, all_functions_dict) for func_expr in func_exprs]
/var/folders/h8/nhgbdr4d18x2r49j4pk5z6gw0000gn/T/ipykernel_30203/2050537243.py:55: RuntimeWarning: coroutine 'search' was never awaited
  func_outputs_2 = [execute_function_by_eval(func_expr) for func_expr in func_exprs]


In [22]:
# first check the openai's function call apis

from openai import OpenAI
from openai.types import FunctionDefinition
from lightrag.utils import setup_env
import json

client = OpenAI()

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris in celsius?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    print(f"response: {response}")
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    print(f"tool_calls: {tool_calls}")
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)# use json.loads to convert a string to a dictionary
            # function_response = function_to_call(
            #     location=function_args.get("location"),
            #     unit=function_args.get("unit"),
            # ) 
            # you have to exactly know the arguments, this does not make sense. How would i know its arguments. **function_args (makes more sense)
            function_response = function_to_call(**function_args)
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
print(run_conversation())

response: ChatCompletion(id='chatcmpl-9eDBpPnQkSDM90VqKgmtGsMJ3k7jJ', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0f6vJCTGXHRDlr3Uns6cczlJ', function=Function(arguments='{"location": "San Francisco, CA", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_GZwXUQ2hVeLmOuf6Ty28JluG', function=Function(arguments='{"location": "Tokyo, Japan", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_ASnPn2pGcyixg7AjCr4DSdYs', function=Function(arguments='{"location": "Paris, France", "unit": "celsius"}', name='get_current_weather'), type='function')]))], created=1719370849, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_4008e3b719', usage=CompletionUsage(completion_tokens=83, prompt_tok

In [23]:
# Function(arguments='{"location": "Tokyo, Japan", "unit": "celsius"}', name='get_current_weather'

There are two important pieces. Getting function schema is not difficult and can be standarized.

The second piece is how to call the function, and how to execute it. The how to call the function depends on how we execute it.

How to execute a function:
1. Eval (LLM will output the code to call the function (in string format))-> Language generation.
2. We manage a function map, and we ask LLm to output either the code string or a structure with the function name and the arguments. We can use the function map to call the function. If its code string, we will have to parse the function call into the name and the arguments. If its a structure, we will have to convert it to data structure that can be used to call the function.

There are just so many different ways to do the actual function call, and different LLM might react differetntly in accuracy to each output format.

Function(arguments='{"location": "Paris, France"}', name='get_current_weather'), type='function')

In [24]:
def get_current_weather(location: str, unit: str = "fahrenheit"):
        """Get the current weather in a given location"""
        if "tokyo" in location.lower():
            return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
        elif "san francisco" in location.lower():
            return json.dumps(
                {"location": "San Francisco", "temperature": "72", "unit": unit}
            )
        elif "paris" in location.lower():
            return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
        else:
            return json.dumps({"location": location, "temperature": "unknown"})



In [ ]:
# v2

from lightrag.core.base_data_class import DataClass
from dataclasses import dataclass, field

@dataclass
class Weather(DataClass):
    location: str = field(metadata={"description": "The city and state, e.g. San Francisco, CA"})
    unit: str = field(metadata={"enum": ["celsius", "fahrenheit"]})

def get_current_weather_2(weather: Weather):
    """Get the current weather in a given location"""
    if "tokyo" in weather.location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": weather.unit})
    elif "san francisco" in weather.location.lower():
        return json.dumps(
            {"location": "San Francisco", "temperature": "72", "unit": weather.unit}
        )
    elif "paris" in weather.location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": weather.unit})
    else:
        return json.dumps({"location": weather.location, "temperature": "unknown"})

In [ ]:
# Create a tool from the class

tool_2 = FunctionTool.from_defaults(fn=get_current_weather_2)

print(tool_2.metadata.to_json())



name: weather, parameter: weather: __main__.Weather    <class '__main__.Weather'>
type_hints[name]: <class '__main__.Weather'>
name: location, parameter: location: str    <class 'str'>
name: unit, parameter: unit: str    <class 'str'>
{
    "name": "get_current_weather_2",
    "description": "get_current_weather_2(weather: __main__.Weather)\nGet the current weather in a given location",
    "parameters": {
        "type": "object",
        "properties": {
            "weather": {
                "type": "Weather",
                "description": "The city and state, e.g. San Francisco, CA",
                "enum": [
                    "celsius",
                    "fahrenheit"
                ]
            }
        },
        "required": [
            "weather"
        ],
        "definitions": {
            "weather": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "str"
                    },
    

Llamaindex



In [ ]:
lightrag_fn_schema =
{
        "type": "object",
        "properties": {
            "weather": {
                "type": "Weather",
                "desc": "The city and state, e.g. San Francisco, CA",
                "enum": [
                    "celsius",
                    "fahrenheit"
                ]
            }
        },
        "required": [
            "weather"
        ],
        "definitions": {
            "weather": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "str"
                    },
                    "unit": {
                        "type": "str"
                    }
                },
                "required": [
                    "location",
                    "unit"
                ]
            }
        }
}

In [ ]:
    llama_fn_schema = {
        "type": "object",
        "properties": {"weather": {"$ref": "#/definitions/Weather"}},
        "required": ["weather"],
        "definitions": {
            "Weather": {
                "title": "Weather",
                "type": "object",
                "properties": {
                    "location": {
                        "title": "Location",
                        "desc": "The city and state, e.g. San Francisco, CA",
                        "type": "string",
                    },
                    "unit": {
                        "title": "Unit",
                        "enum": ["celsius", "fahrenheit"],
                        "type": "string",
                    },
                },
                "required": ["location", "unit"],
                "additionalProperties": false,
            }
        },
    }

In [ ]:
# level 1, call function with default python data types
# such as str, int, float, list, dict, etc.

def _get_current_weather(location: str, unit: str = "fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps(
            {"location": "San Francisco", "temperature": "72", "unit": unit}
        )
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [ ]:
# prepare function tool 
weather_tool = FunctionTool.from_defaults(fn=_get_current_weather)
print(weather_tool)

FunctionTool(metadata=ToolMetadata(name='_get_current_weather', description="_get_current_weather(location: str, unit: str = 'fahrenheit')\nGet the current weather in a given location", parameters={'type': 'object', 'properties': {'location': {'type': 'str'}, 'unit': {'type': 'str', 'default': 'fahrenheit'}}, 'required': ['location']}), fn=<function _get_current_weather at 0x10806ff60>, async_fn=None)


In [ ]:
# prepare a minimal function calling template 
template = r"""<SYS>You have these tools available:
    <TOOLS>
    {% for tool in tools %}
    {{ loop.index }}. ToolName: {{ tool.metadata.name }}
        Tool Description: {{ tool.metadata.description }}
        Tool Parameters: {{ tool.metadata.fn_schema_str }}   
    __________
    {% endfor %}
    </TOOLS>
    {{output_format_str}}
    </SYS>
    User: {{input_str}}
    You:
    """

multiple_function_call_template = r"""<SYS>You can answer user query with these tools:
    <TOOLS>
    {% for tool in tools %}
    {{ loop.index }}. ToolName: {{ tool.metadata.name }}
        Tool Description: {{ tool.metadata.description }}
        Tool Parameters: {{ tool.metadata.fn_schema_str }}   
    __________
    {% endfor %}
    </TOOLS>
    You can call multiple tools by return a list of the following format:
    {{output_format_str}}
    </SYS>
    User: {{input_str}}
    You:
    """

from typing import Dict, Any
from lightrag.core.generator import Generator
from lightrag.core.types import ModelClientType
from lightrag.components.output_parsers import YamlOutputParser

model_kwargs = {"model": "gpt-3.5-turbo", "temperature": 0.3, "stream": False}

@dataclass
class Function(DataClass):
    name: str = field(metadata={"desc": "The name of the function"})
    args: Dict[str, Any] = field(metadata={"desc": "The arguments of the function"})

generator = Generator(
    model_client=ModelClientType.OPENAI(),
    model_kwargs=model_kwargs,
    template=template,
    prompt_kwargs={
        # "tools": [weather_tool],
        "output_format_str": YamlOutputParser(Function).format_instructions(),
        # "output_format_str": Function.to_yaml_signature(),
    },
    output_processors=YamlOutputParser(Function),
)
generator

Generator(
  model_kwargs={'model': 'gpt-3.5-turbo', 'temperature': 0.3, 'stream': False}, 
  (prompt): Prompt(
    template: <SYS>You have these tools available:
        <TOOLS>
        {% for tool in tools %}
        {{ loop.index }}. ToolName: {{ tool.metadata.name }}
            Tool Description: {{ tool.metadata.description }}
            Tool Parameters: {{ tool.metadata.fn_schema_str }}   
        __________
        {% endfor %}
        </TOOLS>
        {{output_format_str}}
        </SYS>
        User: {{input_str}}
        You:
        , prompt_kwargs: {'output_format_str': 'Your output should be formatted as a standard YAML instance with the following schema:\n```\nname: The name of the function (str) (required)\nargs: The arguments of the function (Dict) (required)\n```\n\n-Make sure to always enclose the YAML output in triple backticks (```). Please do not add anything other than valid YAML output!\n-Follow the YAML formatting conventions with an indent of 2 spaces.\n-Quote

In [ ]:
# check the prompt

input_str = "What's the weather like in San Francisco, Tokyo, and Paris in celsius?"

generator.print_prompt(input_str=input_str, tools=[weather_tool])

Prompt:
<SYS>You have these tools available:
    <TOOLS>
    1. ToolName: _get_current_weather
        Tool Description: _get_current_weather(location: str, unit: str = 'fahrenheit')
Get the current weather in a given location
        Tool Parameters: {"type": "object", "properties": {"location": {"type": "str"}, "unit": {"type": "str", "default": "fahrenheit"}}, "required": ["location"]}   
    __________
    </TOOLS>
    Your output should be formatted as a standard YAML instance with the following schema:
```
name: The name of the function (str) (required)
args: The arguments of the function (Dict) (required)
```

-Make sure to always enclose the YAML output in triple backticks (```). Please do not add anything other than valid YAML output!
-Follow the YAML formatting conventions with an indent of 2 spaces.
-Quote the string values properly.

    </SYS>
    User: What's the weather like in San Francisco, Tokyo, and Paris in celsius?
    You:
    


In [ ]:
prompt_kwargs = {
    "input_str": input_str,
    "tools": [weather_tool],
}
output = generator(prompt_kwargs=prompt_kwargs)
structured_output = Function.from_dict(output.data)
print(structured_output)

Function(name='_get_current_weather', args={'location': 'San Francisco', 'unit': 'celsius'})


In [ ]:
# call the function

function_map = {
    "_get_current_weather": weather_tool
}

function_name = structured_output.name
function_args = structured_output.args
function_to_call = function_map[function_name]
function_response = function_to_call(**function_args)
print(function_response)

{"location": "Paris", "temperature": "22", "unit": "celsius"}


# multiple function calls

In [ ]:
generator = Generator(
    model_client=ModelClientType.OPENAI(),
    model_kwargs=model_kwargs,
    template=multiple_function_call_template,
    prompt_kwargs={
        # "tools": [weather_tool],
        "output_format_str": YamlOutputParser(Function).format_instructions(),
        # "output_format_str": Function.to_yaml_signature(),
    },
    output_processors=YamlOutputParser(Function),
)
generator

Generator(
  model_kwargs={'model': 'gpt-3.5-turbo', 'temperature': 0.3, 'stream': False}, 
  (prompt): Prompt(
    template: <SYS>You can answer user query with these tools:
        <TOOLS>
        {% for tool in tools %}
        {{ loop.index }}. ToolName: {{ tool.metadata.name }}
            Tool Description: {{ tool.metadata.description }}
            Tool Parameters: {{ tool.metadata.fn_schema_str }}   
        __________
        {% endfor %}
        </TOOLS>
        You can call multiple tools by return a list of the following format:
        {{output_format_str}}
        </SYS>
        User: {{input_str}}
        You:
        , prompt_kwargs: {'output_format_str': 'Your output should be formatted as a standard YAML instance with the following schema:\n```\nname: The name of the function (str) (required)\nargs: The arguments of the function (Dict) (required)\n```\n\n-Make sure to always enclose the YAML output in triple backticks (```). Please do not add anything other than valid

In [ ]:
# run the query

output = generator(prompt_kwargs=prompt_kwargs)
list_structured_output = [Function.from_dict(item) for item in output.data]
print(output)
print(list_structured_output)

GeneratorOutput(data=[{'name': '_get_current_weather', 'args': {'location': 'San Francisco', 'unit': 'celsius'}}, {'name': '_get_current_weather', 'args': {'location': 'Tokyo', 'unit': 'celsius'}}, {'name': '_get_current_weather', 'args': {'location': 'Paris', 'unit': 'celsius'}}], error=None, usage=None, raw_response='```yaml\n- name: _get_current_weather\n  args:\n    location: "San Francisco"\n    unit: "celsius"\n- name: _get_current_weather\n  args:\n    location: "Tokyo"\n    unit: "celsius"\n- name: _get_current_weather\n  args:\n    location: "Paris"\n    unit: "celsius"\n```')
[Function(name='_get_current_weather', args={'location': 'San Francisco', 'unit': 'celsius'}), Function(name='_get_current_weather', args={'location': 'Tokyo', 'unit': 'celsius'}), Function(name='_get_current_weather', args={'location': 'Paris', 'unit': 'celsius'})]


In [ ]:
for structured_output in list_structured_output:
    function_name = structured_output.name
    function_args = structured_output.args
    function_to_call = function_map[function_name]
    function_response = function_to_call(**function_args)
    print(function_response)

{"location": "San Francisco", "temperature": "72", "unit": "celsius"}
{"location": "Tokyo", "temperature": "10", "unit": "celsius"}
{"location": "Paris", "temperature": "22", "unit": "celsius"}


In [ ]:
from dataclasses import dataclass, field
from typing import Any, Dict

@dataclass
class Address:
    street: str
    city: str
    zipcode: str

@dataclass
class Person:
    name: str
    age: int
    address: Address

# Example instance of the nested dataclasses
person = Person(name="John Doe", age=30, address=Address(street="123 Main St", city="Anytown", zipcode="12345"))
print(person)

def to_dict(obj: Any) -> Dict[str, Any]:
    if hasattr(obj, "__dataclass_fields__"):
        return {key: to_dict(value) for key, value in obj.__dict__.items()}
    elif isinstance(obj, list):
        return [to_dict(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: to_dict(value) for key, value in obj.items()}
    else:
        return obj

# Convert the person instance to a dictionary
person_dict = to_dict(person)
print(person_dict)

Person(name='John Doe', age=30, address=Address(street='123 Main St', city='Anytown', zipcode='12345'))
{'name': 'John Doe', 'age': 30, 'address': {'street': '123 Main St', 'city': 'Anytown', 'zipcode': '12345'}}


In [ ]:
from typing import List
@dataclass
class Address:
    street: str
    city: str
    zipcode: str

@dataclass
class Person:
    name: str
    age: int
    addresses: List[Address]

# Example instance of the nested dataclasses
person = Person(name="John Doe", age=30, addresses=[Address(street="123 Main St", city="Anytown", zipcode="12345"), Address(street="456 Elm St", city="Othertown", zipcode="67890")])
print(person)

Person(name='John Doe', age=30, addresses=[Address(street='123 Main St', city='Anytown', zipcode='12345'), Address(street='456 Elm St', city='Othertown', zipcode='67890')])


In [ ]:
# Convert the person instance to a dictionary
person_dict = to_dict(person)
print(person_dict)

{'name': 'John Doe', 'age': 30, 'addresses': [{'street': '123 Main St', 'city': 'Anytown', 'zipcode': '12345'}, {'street': '456 Elm St', 'city': 'Othertown', 'zipcode': '67890'}]}


In [ ]:
from typing import List, Dict, Optional
def dataclass_obj_to_dict(
    obj: Any, exclude: Optional[Dict[str, List[str]]] = None, parent_key: str = ""
) -> Dict[str, Any]:
    r"""Convert a dataclass object to a dictionary.

    Supports nested dataclasses, lists, and dictionaries.
    Allow exclude keys for each dataclass object.
    Example:

    .. code-block:: python

       from dataclasses import dataclass
       from typing import List

       @dataclass
       class TrecData:
           question: str
           label: int

       @dataclass
       class TrecDataList:

           data: List[TrecData]
           name: str

       trec_data = TrecData(question="What is the capital of France?", label=0)
       trec_data_list = TrecDataList(data=[trec_data], name="trec_data_list")

       dataclass_obj_to_dict(trec_data_list, exclude={"TrecData": ["label"], "TrecDataList": ["name"]})

       # Output:
       # {'data': [{'question': 'What is the capital of France?'}], 'name': 'trec_data_list'}

    """
    if exclude is None:
        exclude = {}

    obj_class_name = obj.__class__.__name__
    current_exclude = exclude.get(obj_class_name, [])

    if hasattr(obj, "__dataclass_fields__"):
        return {
            key: dataclass_obj_to_dict(value, exclude, parent_key=key)
            for key, value in obj.__dict__.items()
            if key not in current_exclude
        }
    elif isinstance(obj, list):
        return [dataclass_obj_to_dict(item, exclude, parent_key) for item in obj]
    elif isinstance(obj, dict):
        return {
            key: dataclass_obj_to_dict(value, exclude, parent_key)
            for key, value in obj.items()
        }
    else:
        return obj

from dataclasses import dataclass
from typing import List

@dataclass
class TrecData:
    question: str
    label: int

@dataclass
class TrecDataList:

    data: List[TrecData]
    name: str

trec_data = TrecData(question="What is the capital of France?", label=0)
trec_data_list = TrecDataList(data=[trec_data], name="trec_data_list")

dataclass_obj_to_dict(trec_data_list, exclude={"TrecData": ["label"], "TrecDataList": ["name"]})

{'data': [{'question': 'What is the capital of France?'}]}

In [ ]:
from typing import Type
def dataclass_obj_from_dict(cls: Type[Any], data: Dict[str, Any]) -> Any:
    if hasattr(cls, "__dataclass_fields__"):
        fieldtypes = {f.name: f.type for f in cls.__dataclass_fields__.values()}
        return cls(**{key: dataclass_obj_from_dict(fieldtypes[key], value) for key, value in data.items()})
    elif isinstance(data, list):
        return [dataclass_obj_from_dict(cls.__args__[0], item) for item in data]
    elif isinstance(data, dict):
        return {key: dataclass_obj_from_dict(cls.__args__[1], value) for key, value in data.items()}
    else:
        return data

In [ ]:
dataclass_obj_from_dict(TrecDataList, dataclass_obj_to_dict(trec_data_list))

TrecDataList(data=[TrecData(question='What is the capital of France?', label=0)], name='trec_data_list')

In [ ]:
dataclass_obj_from_dict(TrecDataList, dataclass_obj_to_dict(trec_data_list, exclude={"TrecData": ["label"], "TrecDataList": ["name"]}))

TypeError: TrecData.__init__() missing 1 required positional argument: 'label'